In [32]:
# Import the CARLA Python API library and some utils
import carla 
import math 
import random 
import time 

client = carla.Client('ines-gpu-01.informatik.uni-mannheim.de', 2001)
client.load_world("Town10HD")
world = client.get_world()
map = world.get_map()
topology = map.get_topology()
wpset = set()
for x in topology:
    wpset.add(x[0])
    wpset.add(x[1])


In [33]:
def get_waypoints(waypoint):
    waypoints = [waypoint]
    nxt = waypoint.next(1)
    if len(nxt) > 0:
        nxt = nxt[0]
        while nxt.road_id == waypoint.road_id:
            waypoints.append(nxt)
            nxt = nxt.next(1)
            if len(nxt) > 0:
                nxt = nxt[0]
            else:
                break
    return waypoints

In [36]:
waypoints_dict = {}
for waypoint in wpset:
    key = "R" +str(waypoint.road_id) + "L" + str(waypoint.lane_id)
    if key not in waypoints_dict:
        waypoints_dict[key] = get_waypoints(waypoint)

In [37]:
from json import JSONEncoder
class WaypointEncoder(JSONEncoder):
    def default(self, waypoint):
        return {
            'x': float('%.3f' % (waypoint.transform.location.x)),
            'y': float('%.3f' % (waypoint.transform.location.y)),
            'rot': float('%.3f' % (waypoint.transform.rotation.yaw)),
        }

In [38]:
import json
with open("Town10HD.json", "w") as f:
    json.dump(waypoints_dict, f, cls=WaypointEncoder)